In [14]:
# importing necassery packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#%matplotlib inline` is a magic command in Jupyter notebooks that allows plots to be displayed directly within the notebook. 

%matplotlib inline
import matplotlib
import warnings
warnings.filterwarnings('ignore')
matplotlib.rcParams["figure.figsize"] = (20,10)

## Data Cleaning
Remove rows with missing values and understanding dataset.There are some data that are ambiguous like room with 42 bedrooms whihc are ambiguous and needs to be removed.

In [15]:
df=pd.read_csv("Bengaluru_House_Data.csv")
print(df.columns)

Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'bath', 'balcony', 'price'],
      dtype='object')


In [16]:
#removing unwanted features
df1=df.drop(['area_type','availability','society','balcony'],axis='columns') # new dataframe after removing unwanted features
df1.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [17]:
# data cleaning by removing row with NaN (Not A Number)
#df1.isnull().sum() to count the number of missing values in each column
# print(df1.isnull().sum())

# defined to extract the numerical value from the 'size' column
def changeToNumber(row):
    return int(row['size'].split(' ')[0])
# removing rows with NaN vallue
df1=df1.dropna(axis=0)
df1.head()
df1['size'].unique()
df1['bhk']=df1.apply(changeToNumber,axis=1)
# print(sorted(df1['bhk'].unique()))
# df1[df1.bhk > 9]['bhk'].count() 
# df1=df1[df1.bhk <9] # removing column where bhk>9 

In [18]:
# total_sqft have values in string format and some values are given in range 
# so remove the data which are corrupted
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True
df1[~df1['total_sqft'].apply(is_float)].shape
#there are 186 record with total_sqft that cannot be converted we need to convert those to float

def convert_total_sqft(col):
    values=col.split("-")
    if len(values)==2:
        return (float(values[0])+float(values[1]))/2
    try:
        return float(col)
    except:
        return None
df2=df1.copy()
df2['total_sqft'] = df2['total_sqft'].apply(convert_total_sqft)
df2=df2.dropna(axis=0)

## Feature Engineering And Dimensional Reduction

In [22]:
# creating new feature price per square feet
df2['price_per_sqft']=df2['price']*100000/df2['total_sqft']
df2.head()

df2.location=df2.location.apply(lambda x:x.strip())
location_stats=df2.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats

location
Whitefield               533
Sarjapur  Road           392
Electronic City          304
Kanakpura Road           264
Thanisandra              235
                        ... 
1 Giri Nagar               1
Kanakapura Road,           1
Kanakapura main  Road      1
Kannur                     1
whitefiled                 1
Name: location, Length: 1287, dtype: int64